In [ ]:
import os
import random
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.applications import ResNet50

In [ ]:
# Define directories for train, test, and validation data
train_dir = '/content/drive/MyDrive/MIS 548 Porject Dataset/Train/'
test_dir = '/content/drive/MyDrive/MIS 548 Porject Dataset/Test/'
val_dir = '/content/drive/MyDrive/MIS 548 Porject Dataset/Validation/'

In [ ]:
# List files in the train directory
print("Files in train directory:")
print(os.listdir(train_dir))

# List files in the test directory
print("Files in test directory:")
print(os.listdir(test_dir))

# List files in the validation directory
print("Files in validation directory:")
print(os.listdir(val_dir))

Files in train directory:
['Fake', 'Real']
Files in test directory:
['Real', 'Fake']
Files in validation directory:
['Fake', 'Real']


In [ ]:
# Count the number of files in each subdirectory of the train directory
train_real_files = len(os.listdir(os.path.join(train_dir, 'Real')))
train_fake_files = len(os.listdir(os.path.join(train_dir, 'Fake')))

# Count the number of files in each subdirectory of the train directory
test_real_files = len(os.listdir(os.path.join(test_dir, 'Real')))
test_fake_files = len(os.listdir(os.path.join(test_dir, 'Fake')))

# Count the number of files in each subdirectory of the train directory
val_real_files = len(os.listdir(os.path.join(val_dir, 'Real')))
val_fake_files = len(os.listdir(os.path.join(val_dir, 'Fake')))

# Print the counts
print("Number of real images in train directory:", train_real_files)
print("Number of fake images in train directory:", train_fake_files)

# Print the counts
print("Number of real images in train directory:", test_real_files)
print("Number of fake images in train directory:", test_fake_files)

# Print the counts
print("Number of real images in train directory:", val_real_files)
print("Number of fake images in train directory:", val_fake_files)


In [ ]:
# Randomly sample 5,000 images from each folder
train_images = random.sample(os.listdir(train_dir), 10000)
test_images = random.sample(os.listdir(test_dir), 2000)
val_images = random.sample(os.listdir(val_dir), 2000)

ValueError: Sample larger than population or is negative

In [ ]:
# Define image dimensions and batch size
img_width, img_height = 224, 224
batch_size = 32

In [ ]:
# Create data generators for train, test, and validation data
train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)
val_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_dataframe(
    dataframe=pd.DataFrame({"filename": train_images}),
    directory=train_dir,
    x_col="filename",
    y_col=None,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode=None,
    shuffle=True
)

test_generator = test_datagen.flow_from_dataframe(
    dataframe=pd.DataFrame({"filename": test_images}),
    directory=test_dir,
    x_col="filename",
    y_col=None,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode=None,
    shuffle=False
)

val_generator = val_datagen.flow_from_dataframe(
    dataframe=pd.DataFrame({"filename": val_images}),
    directory=val_dir,
    x_col="filename",
    y_col=None,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode=None,
    shuffle=False
)

In [ ]:
# Load the pre-trained ResNet50 model without the top classification layer
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(img_width, img_height, 3))

In [ ]:
# Freeze the convolutional layers
for layer in base_model.layers:
    layer.trainable = False

In [ ]:
# Add custom classification layers on top of ResNet50
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu')(x)
predictions = Dense(1, activation='sigmoid')(x)

In [ ]:
# Define the model
model = Model(inputs=base_model.input, outputs=predictions)

In [ ]:
# Compile the model
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [ ]:
# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=len(train_images) // batch_size,
    epochs=10,
    validation_data=val_generator,
    validation_steps=len(val_images) // batch_size
)

In [ ]:
# Evaluate the model on the test data
test_loss, test_accuracy = model.evaluate(test_generator)
print("Test Accuracy:", test_accuracy)

分隔線

In [ ]:
import os
import random
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from sklearn.utils import shuffle

In [ ]:
# Set random seed for reproducibility
random.seed(42)
np.random.seed(42)
tf.random.set_seed(42)

In [ ]:
# Path to the directory containing the dataset
data_dir = "/content/drive/MyDrive/MIS 548 Porject Dataset"

In [ ]:
# Data sampling and loading
def sample_data(directory, sample_size):
    files = os.listdir(directory)
    sampled_files = random.sample(files, sample_size)
    return [os.path.join(directory, file) for file in sampled_files]

def load_data(directory, sample_size):
    real_samples = sample_data(os.path.join(directory, "Real"), sample_size // 2)
    fake_samples = sample_data(os.path.join(directory, "Fake"), sample_size // 2)
    all_samples = real_samples + fake_samples
    labels = [1] * (sample_size // 2) + [0] * (sample_size // 2)
    all_samples, labels = shuffle(all_samples, labels)
    return all_samples, labels

train_samples, train_labels = load_data(os.path.join(data_dir, "Train"), 10000)
val_samples, val_labels = load_data(os.path.join(data_dir, "Validation"), 2000)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Data preprocessing
def preprocess_image(image):
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.resize(image, (224, 224))
    image = tf.keras.applications.resnet.preprocess_input(image)
    return image

def load_and_preprocess_image(path, label):
    image = tf.io.read_file(path)
    return preprocess_image(image), label

In [ ]:
# Create TensorFlow datasets
train_dataset = tf.data.Dataset.from_tensor_slices((train_samples, train_labels))
train_dataset = train_dataset.map(load_and_preprocess_image)
train_dataset = train_dataset.shuffle(buffer_size=len(train_samples)).batch(32)

val_dataset = tf.data.Dataset.from_tensor_slices((val_samples, val_labels))
val_dataset = val_dataset.map(load_and_preprocess_image)
val_dataset = val_dataset.batch(32)

In [ ]:
# Define the ResNet model architecture
base_model = ResNet50(weights='imagenet', include_top=False)
x = base_model.output
x = GlobalAveragePooling2D()(x)
predictions = Dense(1, activation='sigmoid')(x)
model = Model(inputs=base_model.input, outputs=predictions)

In [ ]:
# Compile the model
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [ ]:
# Train the model
model.fit(train_dataset,
          epochs=10,
          validation_data=val_dataset)

Epoch 1/10


In [ ]:
# Save the model
model.save("resnet_model.h5")

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
# Evaluate the model
test_samples, test_labels = load_data(os.path.join(data_dir, "Test"), 2000)
test_dataset = tf.data.Dataset.from_tensor_slices((test_samples, test_labels))
test_dataset = test_dataset.map(load_and_preprocess_image)
test_dataset = test_dataset.batch(32)

loss, accuracy = model.evaluate(test_dataset)
print("Test Loss:", loss)
print("Test Accuracy:", accuracy)

63/63 [==============================] - 189s 3s/step - loss: 0.9597 - accuracy: 0.7230
Test Loss: 0.9596732258796692
Test Accuracy: 0.7229999899864197


分隔線 - ResNet 參數調整 1

In [ ]:
import os
import random
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import LearningRateScheduler
from sklearn.utils import shuffle

In [ ]:
# Set random seed for reproducibility
random.seed(42)
np.random.seed(42)
tf.random.set_seed(42)

In [ ]:
# Path to the directory containing the dataset
data_dir = "/content/drive/MyDrive/MIS 548 Porject Dataset"

In [ ]:
# Data sampling and loading
def sample_data(directory, sample_size):
    files = os.listdir(directory)
    sampled_files = random.sample(files, sample_size)
    return [os.path.join(directory, file) for file in sampled_files]

def load_data(directory, sample_size):
    real_samples = sample_data(os.path.join(directory, "Real"), sample_size // 2)
    fake_samples = sample_data(os.path.join(directory, "Fake"), sample_size // 2)
    all_samples = real_samples + fake_samples
    labels = [1] * (sample_size // 2) + [0] * (sample_size // 2)
    all_samples, labels = shuffle(all_samples, labels)
    return all_samples, labels

train_samples, train_labels = load_data(os.path.join(data_dir, "Train"), 10000)
val_samples, val_labels = load_data(os.path.join(data_dir, "Validation"), 2000)

In [ ]:
# Data preprocessing and augmentation
train_datagen = ImageDataGenerator(
    preprocessing_function=tf.keras.applications.resnet.preprocess_input,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

train_dataset = train_datagen.flow_from_directory(
    os.path.join(data_dir, "Train"),
    #target_size=(224, 224),
    batch_size=32,
    class_mode='binary',
    shuffle=True
)

val_datagen = ImageDataGenerator(
    preprocessing_function=tf.keras.applications.resnet.preprocess_input
)

val_dataset = val_datagen.flow_from_directory(
    os.path.join(data_dir, "Validation"),
    #target_size=(224, 224),
    batch_size=32,
    class_mode='binary',
    shuffle=False
)

Found 140110 images belonging to 2 classes.
Found 39428 images belonging to 2 classes.


In [ ]:
# Define the ResNet model architecture
base_model = ResNet50(weights='imagenet', include_top=False)
x = base_model.output
x = GlobalAveragePooling2D()(x)
predictions = Dense(1, activation='sigmoid')(x)
model = Model(inputs=base_model.input, outputs=predictions)

In [ ]:
# Fine-tune the model
for layer in base_model.layers:
    layer.trainable = False

In [ ]:
# Compile the model with a custom learning rate scheduler
initial_learning_rate = 0.001
epochs = 10

lr_schedule = LearningRateScheduler(lambda epoch: initial_learning_rate * 0.9 ** epoch)

model.compile(optimizer=Adam(learning_rate=initial_learning_rate),
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [ ]:
# Train the model
model.fit(train_dataset,
          epochs=epochs,
          validation_data=val_dataset,
          callbacks=[lr_schedule])

Epoch 1/10
 577/4379 [==>...........................] - ETA: 25:48:20 - loss: 0.5019 - accuracy: 0.7506

In [ ]:
# Evaluate the model
test_datagen = ImageDataGenerator(
    preprocessing_function=tf.keras.applications.resnet.preprocess_input
)

test_dataset = test_datagen.flow_from_directory(
    os.path.join(data_dir, "Test"),
    #target_size=(224, 224),
    batch_size=32,
    class_mode='binary',
    shuffle=False
)

loss, accuracy = model.evaluate(test_dataset)
print("Test Loss:", loss)
print("Test Accuracy:", accuracy)


分隔線 - ResNet 參數調整 2

In [ ]:
import os
import random
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import LearningRateScheduler, EarlyStopping
from sklearn.utils import shuffle

In [ ]:
# Set random seed for reproducibility
random.seed(42)
np.random.seed(42)
tf.random.set_seed(42)

In [ ]:
# Path to the directory containing the dataset
data_dir = "/content/drive/MyDrive/MIS 548 Porject Dataset"

In [ ]:
# Data sampling and loading
def sample_data(directory, sample_size):
    files = os.listdir(directory)
    sampled_files = random.sample(files, sample_size)
    return [os.path.join(directory, file) for file in sampled_files]

def load_data(directory, sample_size):
    real_samples = sample_data(os.path.join(directory, "Real"), sample_size // 2)
    fake_samples = sample_data(os.path.join(directory, "Fake"), sample_size // 2)
    all_samples = real_samples + fake_samples
    labels = [1] * (sample_size // 2) + [0] * (sample_size // 2)
    all_samples, labels = shuffle(all_samples, labels)
    return all_samples, labels

train_samples, train_labels = load_data(os.path.join(data_dir, "Train"), 10000)
val_samples, val_labels = load_data(os.path.join(data_dir, "Validation"), 2000)

In [ ]:
# Data preprocessing and augmentation
train_datagen = ImageDataGenerator(
    preprocessing_function=tf.keras.applications.resnet.preprocess_input,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

train_dataset = train_datagen.flow_from_directory(
    os.path.join(data_dir, "Train"),
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary',
    shuffle=True
)

val_datagen = ImageDataGenerator(
    preprocessing_function=tf.keras.applications.resnet.preprocess_input
)

val_dataset = val_datagen.flow_from_directory(
    os.path.join(data_dir, "Validation"),
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary',
    shuffle=False
)

Found 140110 images belonging to 2 classes.
Found 39428 images belonging to 2 classes.


In [ ]:
# Define the ResNet model architecture
base_model = ResNet50(weights='imagenet', include_top=False)
x = base_model.output
x = GlobalAveragePooling2D()(x)
predictions = Dense(1, activation='sigmoid')(x)
model = Model(inputs=base_model.input, outputs=predictions)

In [ ]:
# Fine-tune the model
for layer in base_model.layers:
    layer.trainable = False

In [ ]:
# Compile the model with a custom learning rate scheduler and early stopping
initial_learning_rate = 0.001
epochs = 5

lr_schedule = LearningRateScheduler(lambda epoch: initial_learning_rate * 0.9 ** epoch)
early_stopping = EarlyStopping(monitor='val_loss', patience=2, restore_best_weights=True)

model.compile(optimizer=Adam(learning_rate=initial_learning_rate),
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [ ]:
# Train the model
history = model.fit(train_dataset,
                    epochs=epochs,
                    validation_data=val_dataset,
                    callbacks=[lr_schedule, early_stopping])

Epoch 1/5
 673/4379 [===>..........................] - ETA: 4:01:38 - loss: 0.4915 - accuracy: 0.7550

KeyboardInterrupt: 

In [ ]:
# Evaluate the model
test_datagen = ImageDataGenerator(
    preprocessing_function=tf.keras.applications.resnet.preprocess_input
)

test_dataset = test_datagen.flow_from_directory(
    os.path.join(data_dir, "Test"),
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary',
    shuffle=False
)

loss, accuracy = model.evaluate(test_dataset)
print("Test Loss:", loss)
print("Test Accuracy:", accuracy)

VGG 16

In [ ]:
import os
import random
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from sklearn.utils import shuffle

In [ ]:
# Set random seed for reproducibility
random.seed(42)
np.random.seed(42)
tf.random.set_seed(42)

In [ ]:
# Path to the directory containing the dataset
data_dir = "/content/drive/MyDrive/MIS 548 Porject Dataset"

In [ ]:
# Data sampling and loading
def sample_data(directory, sample_size):
    files = os.listdir(directory)
    sampled_files = random.sample(files, sample_size)
    return [os.path.join(directory, file) for file in sampled_files]

def load_data(directory, sample_size):
    real_samples = sample_data(os.path.join(directory, "Real"), sample_size // 2)
    fake_samples = sample_data(os.path.join(directory, "Fake"), sample_size // 2)
    all_samples = real_samples + fake_samples
    labels = [1] * (sample_size // 2) + [0] * (sample_size // 2)
    all_samples, labels = shuffle(all_samples, labels)
    return all_samples, labels

train_samples, train_labels = load_data(os.path.join(data_dir, "Train"), 10000)
val_samples, val_labels = load_data(os.path.join(data_dir, "Validation"), 2000)

In [ ]:
# Data preprocessing
def preprocess_image(image):
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.resize(image, (224, 224))
    image = tf.keras.applications.vgg16.preprocess_input(image)
    return image

def load_and_preprocess_image(path, label):
    image = tf.io.read_file(path)
    return preprocess_image(image), label

In [ ]:
# Create TensorFlow datasets
train_dataset = tf.data.Dataset.from_tensor_slices((train_samples, train_labels))
train_dataset = train_dataset.map(load_and_preprocess_image)
train_dataset = train_dataset.shuffle(buffer_size=len(train_samples)).batch(32)

val_dataset = tf.data.Dataset.from_tensor_slices((val_samples, val_labels))
val_dataset = val_dataset.map(load_and_preprocess_image)
val_dataset = val_dataset.batch(32)

In [ ]:
# Define the VGG16 model architecture
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
x = base_model.output
x = GlobalAveragePooling2D()(x)
predictions = Dense(1, activation='sigmoid')(x)
model = Model(inputs=base_model.input, outputs=predictions)

In [ ]:
# Compile the model
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [ ]:
# Train the model
model.fit(train_dataset,
          epochs=5,
          validation_data=val_dataset)

Epoch 1/5
313/313 [==============================] - 395s 726ms/step - loss: 1.1026 - accuracy: 0.5008 - val_loss: 0.6933 - val_accuracy: 0.5000
Epoch 2/5
313/313 [==============================] - 68s 142ms/step - loss: 0.6939 - accuracy: 0.5087 - val_loss: 0.6948 - val_accuracy: 0.5010
Epoch 3/5
313/313 [==============================] - 554s 2s/step - loss: 0.6922 - accuracy: 0.5232 - val_loss: 0.6930 - val_accuracy: 0.5080
Epoch 4/5
313/313 [==============================] - 60s 136ms/step - loss: 0.6917 - accuracy: 0.5262 - val_loss: 0.6932 - val_accuracy: 0.5000
Epoch 5/5
313/313 [==============================] - 60s 136ms/step - loss: 0.6932 - accuracy: 0.5000 - val_loss: 0.6932 - val_accuracy: 0.5000


In [ ]:
# Save the model
model.save("vgg16_model.h5")

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
# Evaluate the model
test_samples, test_labels = load_data(os.path.join(data_dir, "Test"), 2000)
test_dataset = tf.data.Dataset.from_tensor_slices((test_samples, test_labels))
test_dataset = test_dataset.map(load_and_preprocess_image)
test_dataset = test_dataset.batch(32)

loss, accuracy = model.evaluate(test_dataset)
print("Test Loss:", loss)
print("Test Accuracy:", accuracy)


63/63 [==============================] - 182s 3s/step - loss: 0.6932 - accuracy: 0.5000
Test Loss: 0.6931784152984619
Test Accuracy: 0.5


ResNet 101




In [ ]:
import os
import random
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet101
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from sklearn.utils import shuffle

In [ ]:
# Set random seed for reproducibility
random.seed(42)
np.random.seed(42)
tf.random.set_seed(42)

In [ ]:
# Path to the directory containing the dataset
data_dir = "/content/drive/MyDrive/MIS 548 Porject Dataset"

In [ ]:
# Data sampling and loading
def sample_data(directory, sample_size):
    files = os.listdir(directory)
    sampled_files = random.sample(files, sample_size)
    return [os.path.join(directory, file) for file in sampled_files]

def load_data(directory, sample_size):
    real_samples = sample_data(os.path.join(directory, "Real"), sample_size // 2)
    fake_samples = sample_data(os.path.join(directory, "Fake"), sample_size // 2)
    all_samples = real_samples + fake_samples
    labels = [1] * (sample_size // 2) + [0] * (sample_size // 2)
    all_samples, labels = shuffle(all_samples, labels)
    return all_samples, labels

train_samples, train_labels = load_data(os.path.join(data_dir, "Train"), 10000)
val_samples, val_labels = load_data(os.path.join(data_dir, "Validation"), 2000)

In [ ]:
# Data preprocessing
def preprocess_image(image):
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.resize(image, (224, 224))
    image = tf.keras.applications.resnet.preprocess_input(image)
    return image

def load_and_preprocess_image(path, label):
    image = tf.io.read_file(path)
    return preprocess_image(image), label

In [ ]:
# Create TensorFlow datasets
train_dataset = tf.data.Dataset.from_tensor_slices((train_samples, train_labels))
train_dataset = train_dataset.map(load_and_preprocess_image)
train_dataset = train_dataset.shuffle(buffer_size=len(train_samples)).batch(32)

val_dataset = tf.data.Dataset.from_tensor_slices((val_samples, val_labels))
val_dataset = val_dataset.map(load_and_preprocess_image)
val_dataset = val_dataset.batch(32)

In [ ]:
# Define the ResNet101 model architecture
base_model = ResNet101(weights='imagenet', include_top=False)
x = base_model.output
x = GlobalAveragePooling2D()(x)
predictions = Dense(1, activation='sigmoid')(x)
model = Model(inputs=base_model.input, outputs=predictions)

171446536/171446536 [==============================] - 4s 0us/step


In [ ]:
# Compile the model
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [ ]:
# Train the model
model.fit(train_dataset,
          epochs=5,
          validation_data=val_dataset)

Epoch 1/5
313/313 [==============================] - 2132s 7s/step - loss: 0.3446 - accuracy: 0.8598 - val_loss: 0.6468 - val_accuracy: 0.7580
Epoch 2/5
313/313 [==============================] - 79s 193ms/step - loss: 0.1697 - accuracy: 0.9307 - val_loss: 1.1761 - val_accuracy: 0.7610
Epoch 3/5
313/313 [==============================] - 78s 193ms/step - loss: 0.1277 - accuracy: 0.9484 - val_loss: 0.5023 - val_accuracy: 0.9075
Epoch 4/5
313/313 [==============================] - 78s 192ms/step - loss: 0.1086 - accuracy: 0.9577 - val_loss: 0.3187 - val_accuracy: 0.9015
Epoch 5/5
313/313 [==============================] - 78s 192ms/step - loss: 0.0946 - accuracy: 0.9623 - val_loss: 0.2026 - val_accuracy: 0.9300


In [ ]:
# Save the model
model.save("resnet101_model.h5")

In [ ]:
# Evaluate the model
test_samples, test_labels = load_data(os.path.join(data_dir, "Test"), 2000)
test_dataset = tf.data.Dataset.from_tensor_slices((test_samples, test_labels))
test_dataset = test_dataset.map(load_and_preprocess_image)
test_dataset = test_dataset.batch(32)

loss, accuracy = model.evaluate(test_dataset)
print("Test Loss:", loss)
print("Test Accuracy:", accuracy)

63/63 [==============================] - 70s 1s/step - loss: 0.3668 - accuracy: 0.8700
Test Loss: 0.36679160594940186
Test Accuracy: 0.8700000047683716


Inception V3

In [ ]:
import os
import random
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from sklearn.utils import shuffle

In [ ]:
# Set random seed for reproducibility
random.seed(42)
np.random.seed(42)
tf.random.set_seed(42)

In [ ]:
# Path to the directory containing the dataset
data_dir = "/content/drive/MyDrive/MIS 548 Porject Dataset"

In [ ]:
# Data sampling and loading
def sample_data(directory, sample_size):
    files = os.listdir(directory)
    sampled_files = random.sample(files, sample_size)
    return [os.path.join(directory, file) for file in sampled_files]

def load_data(directory, sample_size):
    real_samples = sample_data(os.path.join(directory, "Real"), sample_size // 2)
    fake_samples = sample_data(os.path.join(directory, "Fake"), sample_size // 2)
    all_samples = real_samples + fake_samples
    labels = [1] * (sample_size // 2) + [0] * (sample_size // 2)
    all_samples, labels = shuffle(all_samples, labels)
    return all_samples, labels

train_samples, train_labels = load_data(os.path.join(data_dir, "Train"), 10000)
val_samples, val_labels = load_data(os.path.join(data_dir, "Validation"), 2000)

In [ ]:
# Data preprocessing
def preprocess_image(image):
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.resize(image, (299, 299))  # InceptionV3 input size
    image = tf.keras.applications.inception_v3.preprocess_input(image)
    return image

def load_and_preprocess_image(path, label):
    image = tf.io.read_file(path)
    return preprocess_image(image), label

In [ ]:
# Create TensorFlow datasets
train_dataset = tf.data.Dataset.from_tensor_slices((train_samples, train_labels))
train_dataset = train_dataset.map(load_and_preprocess_image)
train_dataset = train_dataset.shuffle(buffer_size=len(train_samples)).batch(32)

val_dataset = tf.data.Dataset.from_tensor_slices((val_samples, val_labels))
val_dataset = val_dataset.map(load_and_preprocess_image)
val_dataset = val_dataset.batch(32)

In [ ]:
# Define the GoogLeNet (InceptionV3) model architecture
base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=(299, 299, 3))
x = base_model.output
x = GlobalAveragePooling2D()(x)
predictions = Dense(1, activation='sigmoid')(x)
model = Model(inputs=base_model.input, outputs=predictions)

87910968/87910968 [==============================] - 3s 0us/step


In [ ]:
# Compile the model
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [ ]:
# Train the model
model.fit(train_dataset,
          epochs=5,
          validation_data=val_dataset)

Epoch 1/5
313/313 [==============================] - 948s 176ms/step - loss: 0.2010 - accuracy: 0.9127 - val_loss: 0.3440 - val_accuracy: 0.9010
Epoch 2/5
313/313 [==============================] - 68s 164ms/step - loss: 0.1058 - accuracy: 0.9575 - val_loss: 0.4894 - val_accuracy: 0.8080
Epoch 3/5
313/313 [==============================] - 67s 164ms/step - loss: 0.0823 - accuracy: 0.9674 - val_loss: 0.2343 - val_accuracy: 0.9115
Epoch 4/5
313/313 [==============================] - 68s 164ms/step - loss: 0.0736 - accuracy: 0.9709 - val_loss: 0.3942 - val_accuracy: 0.8550
Epoch 5/5
313/313 [==============================] - 68s 164ms/step - loss: 0.0504 - accuracy: 0.9789 - val_loss: 0.1590 - val_accuracy: 0.9510


In [ ]:
# Save the model
model.save("inceptionv3_model.h5")

In [ ]:
# Evaluate the model
test_samples, test_labels = load_data(os.path.join(data_dir, "Test"), 2000)
test_dataset = tf.data.Dataset.from_tensor_slices((test_samples, test_labels))
test_dataset = test_dataset.map(load_and_preprocess_image)
test_dataset = test_dataset.batch(32)

loss, accuracy = model.evaluate(test_dataset)
print("Test Loss:", loss)
print("Test Accuracy:", accuracy)


63/63 [==============================] - 68s 1s/step - loss: 0.4633 - accuracy: 0.8670
Test Loss: 0.4633050262928009
Test Accuracy: 0.8669999837875366


EfficientNet

In [ ]:
import os
import random
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from sklearn.utils import shuffle

In [ ]:
# Set random seed for reproducibility
random.seed(42)
np.random.seed(42)
tf.random.set_seed(42)

In [ ]:
# Path to the directory containing the dataset
data_dir = "/content/drive/MyDrive/MIS 548 Porject Dataset"

In [ ]:
# Data sampling and loading
def sample_data(directory, sample_size):
    files = os.listdir(directory)
    sampled_files = random.sample(files, sample_size)
    return [os.path.join(directory, file) for file in sampled_files]

def load_data(directory, sample_size):
    real_samples = sample_data(os.path.join(directory, "Real"), sample_size // 2)
    fake_samples = sample_data(os.path.join(directory, "Fake"), sample_size // 2)
    all_samples = real_samples + fake_samples
    labels = [1] * (sample_size // 2) + [0] * (sample_size // 2)
    all_samples, labels = shuffle(all_samples, labels)
    return all_samples, labels

train_samples, train_labels = load_data(os.path.join(data_dir, "Train"), 10000)
val_samples, val_labels = load_data(os.path.join(data_dir, "Validation"), 2000)

In [ ]:
# Data preprocessing
def preprocess_image(image):
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.resize(image, (224, 224))  # EfficientNet input size
    image = tf.keras.applications.efficientnet.preprocess_input(image)
    return image

def load_and_preprocess_image(path, label):
    image = tf.io.read_file(path)
    return preprocess_image(image), label

In [ ]:
# Create TensorFlow datasets
train_dataset = tf.data.Dataset.from_tensor_slices((train_samples, train_labels))
train_dataset = train_dataset.map(load_and_preprocess_image)
train_dataset = train_dataset.shuffle(buffer_size=len(train_samples)).batch(32)

val_dataset = tf.data.Dataset.from_tensor_slices((val_samples, val_labels))
val_dataset = val_dataset.map(load_and_preprocess_image)
val_dataset = val_dataset.batch(32)

In [ ]:
# Define the EfficientNet model architecture
base_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
x = base_model.output
x = GlobalAveragePooling2D()(x)
predictions = Dense(1, activation='sigmoid')(x)
model = Model(inputs=base_model.input, outputs=predictions)

16705208/16705208 [==============================] - 1s 0us/step


In [ ]:
# Compile the model
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [ ]:
# Train the model
model.fit(train_dataset,
          epochs=5,
          validation_data=val_dataset)

Epoch 1/5
313/313 [==============================] - 1080s 652ms/step - loss: 0.1629 - accuracy: 0.9345 - val_loss: 0.1394 - val_accuracy: 0.9490
Epoch 2/5
313/313 [==============================] - 51s 116ms/step - loss: 0.0657 - accuracy: 0.9720 - val_loss: 0.1614 - val_accuracy: 0.9405
Epoch 3/5
313/313 [==============================] - 51s 116ms/step - loss: 0.0483 - accuracy: 0.9806 - val_loss: 0.1565 - val_accuracy: 0.9480
Epoch 4/5
313/313 [==============================] - 52s 117ms/step - loss: 0.0414 - accuracy: 0.9840 - val_loss: 0.1289 - val_accuracy: 0.9575
Epoch 5/5
313/313 [==============================] - 52s 116ms/step - loss: 0.0352 - accuracy: 0.9867 - val_loss: 0.1778 - val_accuracy: 0.9405


In [ ]:
# Save the model
model.save("efficientnet_model.h5")

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
# Evaluate the model
test_samples, test_labels = load_data(os.path.join(data_dir, "Test"), 2000)
test_dataset = tf.data.Dataset.from_tensor_slices((test_samples, test_labels))
test_dataset = test_dataset.map(load_and_preprocess_image)
test_dataset = test_dataset.batch(32)

loss, accuracy = model.evaluate(test_dataset)
print("Test Loss:", loss)
print("Test Accuracy:", accuracy)

63/63 [==============================] - 219s 3s/step - loss: 0.3738 - accuracy: 0.8810
Test Loss: 0.37379613518714905
Test Accuracy: 0.8809999823570251


Based on the provided output, here are some insights that can help identify which technique might be more suitable:

**Training and Validation Performance:**

The training accuracy starts at 93.45% and increases to 98.67% over the epochs.
The validation accuracy starts at 94.90% and fluctuates slightly but remains relatively high, ranging between 94.05% and 95.75%.
This indicates that the model is learning well from the training data and generalizing reasonably well to unseen validation data.

**Test Performance:**

The test accuracy is 88.10%, which is lower than the validation accuracy.
This suggests that there might be some overfitting occurring, as the model is not performing as well on unseen test data as it did on the validation set.
Given these insights, the following techniques may be suitable to improve test accuracy:

* Regularization: Since the training accuracy is significantly higher than the test accuracy, it indicates that the model might be overfitting. Regularization techniques such as dropout or weight decay can help reduce overfitting and improve generalization to the test set.
* Data Augmentation: If the validation accuracy remains stable but the test accuracy is significantly lower, it might suggest that the model is not exposed to enough variations in the training data. Data augmentation techniques can help introduce more diversity into the training data, potentially leading to better generalization to unseen test data.
* Learning Rate Scheduling: If the training and validation accuracy plateaus or fluctuates significantly, adjusting the learning rate schedule may help stabilize the training process and improve convergence, which can indirectly lead to better test accuracy.

Given these insights, a combination of regularization techniques and data augmentation may be particularly effective in improving test accuracy while maintaining the same number of sampling data.

EfficientNetB1

In [ ]:
import os
import random
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import EfficientNetB1  # Change import statement
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from sklearn.utils import shuffle

In [ ]:
# Set random seed for reproducibility
random.seed(42)
np.random.seed(42)
tf.random.set_seed(42)

In [ ]:
# Path to the directory containing the dataset
data_dir = "/content/drive/MyDrive/MIS 548 Porject Dataset"

In [ ]:
# Data sampling and loading
def sample_data(directory, sample_size):
    files = os.listdir(directory)
    sampled_files = random.sample(files, sample_size)
    return [os.path.join(directory, file) for file in sampled_files]

def load_data(directory, sample_size):
    real_samples = sample_data(os.path.join(directory, "Real"), sample_size // 2)
    fake_samples = sample_data(os.path.join(directory, "Fake"), sample_size // 2)
    all_samples = real_samples + fake_samples
    labels = [1] * (sample_size // 2) + [0] * (sample_size // 2)
    all_samples, labels = shuffle(all_samples, labels)
    return all_samples, labels

train_samples, train_labels = load_data(os.path.join(data_dir, "Train"), 10000)
val_samples, val_labels = load_data(os.path.join(data_dir, "Validation"), 2000)

In [ ]:
# Data preprocessing
def preprocess_image(image):
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.resize(image, (224, 224))  # EfficientNet input size
    image = tf.keras.applications.efficientnet.preprocess_input(image)
    return image

def load_and_preprocess_image(path, label):
    image = tf.io.read_file(path)
    return preprocess_image(image), label

In [ ]:
# Create TensorFlow datasets
train_dataset = tf.data.Dataset.from_tensor_slices((train_samples, train_labels))
train_dataset = train_dataset.map(load_and_preprocess_image)
train_dataset = train_dataset.shuffle(buffer_size=len(train_samples)).batch(32)

val_dataset = tf.data.Dataset.from_tensor_slices((val_samples, val_labels))
val_dataset = val_dataset.map(load_and_preprocess_image)
val_dataset = val_dataset.batch(32)

In [ ]:
# Define the EfficientNetB1 model architecture
base_model = EfficientNetB1(weights='imagenet', include_top=False)  # Change model architecture
x = base_model.output
x = GlobalAveragePooling2D()(x)
predictions = Dense(1, activation='sigmoid')(x)
model = Model(inputs=base_model.input, outputs=predictions)

27018416/27018416 [==============================] - 1s 0us/step


In [ ]:
# Compile the model
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [ ]:
# Train the model
model.fit(train_dataset,
          epochs=5,
          validation_data=val_dataset)

Epoch 1/5
313/313 [==============================] - 677s 2s/step - loss: 0.1659 - accuracy: 0.9365 - val_loss: 0.2945 - val_accuracy: 0.8950
Epoch 2/5
313/313 [==============================] - 62s 152ms/step - loss: 0.0750 - accuracy: 0.9716 - val_loss: 0.1333 - val_accuracy: 0.9570
Epoch 3/5
313/313 [==============================] - 62s 153ms/step - loss: 0.0506 - accuracy: 0.9801 - val_loss: 0.1972 - val_accuracy: 0.9345
Epoch 4/5
313/313 [==============================] - 62s 153ms/step - loss: 0.0379 - accuracy: 0.9838 - val_loss: 0.1561 - val_accuracy: 0.9450
Epoch 5/5
313/313 [==============================] - 62s 154ms/step - loss: 0.0370 - accuracy: 0.9860 - val_loss: 0.2246 - val_accuracy: 0.9400


In [ ]:
# Save the model
model.save("efficientnetB1_model.h5")

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
# Evaluate the model
test_samples, test_labels = load_data(os.path.join(data_dir, "Test"), 2000)
test_dataset = tf.data.Dataset.from_tensor_slices((test_samples, test_labels))
test_dataset = test_dataset.map(load_and_preprocess_image)
test_dataset = test_dataset.batch(32)

loss, accuracy = model.evaluate(test_dataset)
print("Test Loss:", loss)
print("Test Accuracy:", accuracy)


63/63 [==============================] - 179s 3s/step - loss: 0.8707 - accuracy: 0.8550
Test Loss: 0.8706620335578918
Test Accuracy: 0.8550000190734863


EfficientNetB2

In [ ]:
import os
import random
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import EfficientNetB2  # Change import statement
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from sklearn.utils import shuffle

In [ ]:
# Set random seed for reproducibility
random.seed(42)
np.random.seed(42)
tf.random.set_seed(42)

In [ ]:
# Path to the directory containing the dataset
data_dir = "/content/drive/MyDrive/MIS 548 Porject Dataset"

In [ ]:
# Data sampling and loading
def sample_data(directory, sample_size):
    files = os.listdir(directory)
    sampled_files = random.sample(files, sample_size)
    return [os.path.join(directory, file) for file in sampled_files]

def load_data(directory, sample_size):
    real_samples = sample_data(os.path.join(directory, "Real"), sample_size // 2)
    fake_samples = sample_data(os.path.join(directory, "Fake"), sample_size // 2)
    all_samples = real_samples + fake_samples
    labels = [1] * (sample_size // 2) + [0] * (sample_size // 2)
    all_samples, labels = shuffle(all_samples, labels)
    return all_samples, labels

train_samples, train_labels = load_data(os.path.join(data_dir, "Train"), 10000)
val_samples, val_labels = load_data(os.path.join(data_dir, "Validation"), 2000)

In [ ]:
# Data preprocessing
def preprocess_image(image):
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.resize(image, (224, 224))
    image = tf.keras.applications.efficientnet.preprocess_input(image)  # Change preprocessing function
    return image

def load_and_preprocess_image(path, label):
    image = tf.io.read_file(path)
    return preprocess_image(image), label

In [ ]:
# Create TensorFlow datasets
train_dataset = tf.data.Dataset.from_tensor_slices((train_samples, train_labels))
train_dataset = train_dataset.map(load_and_preprocess_image)
train_dataset = train_dataset.shuffle(buffer_size=len(train_samples)).batch(32)

val_dataset = tf.data.Dataset.from_tensor_slices((val_samples, val_labels))
val_dataset = val_dataset.map(load_and_preprocess_image)
val_dataset = val_dataset.batch(32)

In [ ]:
# Define the EfficientNetB2 model architecture
base_model = EfficientNetB2(weights='imagenet', include_top=False)  # Change model architecture
x = base_model.output
x = GlobalAveragePooling2D()(x)
predictions = Dense(1, activation='sigmoid')(x)
model = Model(inputs=base_model.input, outputs=predictions)

31790344/31790344 [==============================] - 1s 0us/step


In [ ]:
# Compile the model
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [ ]:
# Train the model
model.fit(train_dataset,
          epochs=5,
          validation_data=val_dataset)

Epoch 1/5
313/313 [==============================] - 1731s 165ms/step - loss: 0.1651 - accuracy: 0.9345 - val_loss: 0.2171 - val_accuracy: 0.9300
Epoch 2/5
313/313 [==============================] - 64s 158ms/step - loss: 0.0732 - accuracy: 0.9706 - val_loss: 0.1406 - val_accuracy: 0.9465
Epoch 3/5
313/313 [==============================] - 64s 159ms/step - loss: 0.0511 - accuracy: 0.9804 - val_loss: 0.1395 - val_accuracy: 0.9485
Epoch 4/5
313/313 [==============================] - 64s 158ms/step - loss: 0.0377 - accuracy: 0.9851 - val_loss: 0.1543 - val_accuracy: 0.9535
Epoch 5/5
313/313 [==============================] - 64s 158ms/step - loss: 0.0272 - accuracy: 0.9902 - val_loss: 0.2365 - val_accuracy: 0.9295


In [ ]:
# Save the model
model.save("efficientnet_b2_model.h5")

In [ ]:
# Evaluate the model
test_samples, test_labels = load_data(os.path.join(data_dir, "Test"), 2000)
test_dataset = tf.data.Dataset.from_tensor_slices((test_samples, test_labels))
test_dataset = test_dataset.map(load_and_preprocess_image)
test_dataset = test_dataset.batch(32)

loss, accuracy = model.evaluate(test_dataset)
print("Test Loss:", loss)
print("Test Accuracy:", accuracy)


63/63 [==============================] - 64s 1s/step - loss: 0.3937 - accuracy: 0.8770
Test Loss: 0.393665075302124
Test Accuracy: 0.8769999742507935


EfficientNetB3

In [ ]:
import os
import random
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import EfficientNetB3  # Update import statement
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from sklearn.utils import shuffle

In [ ]:
# Set random seed for reproducibility
random.seed(42)
np.random.seed(42)
tf.random.set_seed(42)

In [ ]:
# Path to the directory containing the dataset
data_dir = "/content/drive/MyDrive/MIS 548 Porject Dataset"

In [ ]:
# Data sampling and loading
def sample_data(directory, sample_size):
    files = os.listdir(directory)
    sampled_files = random.sample(files, sample_size)
    return [os.path.join(directory, file) for file in sampled_files]

def load_data(directory, sample_size):
    real_samples = sample_data(os.path.join(directory, "Real"), sample_size // 2)
    fake_samples = sample_data(os.path.join(directory, "Fake"), sample_size // 2)
    all_samples = real_samples + fake_samples
    labels = [1] * (sample_size // 2) + [0] * (sample_size // 2)
    all_samples, labels = shuffle(all_samples, labels)
    return all_samples, labels

train_samples, train_labels = load_data(os.path.join(data_dir, "Train"), 10000)
val_samples, val_labels = load_data(os.path.join(data_dir, "Validation"), 2000)

In [ ]:
# Data preprocessing
def preprocess_image(image):
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.resize(image, (224, 224))
    image = tf.keras.applications.efficientnet.preprocess_input(image)  # Update preprocessing function
    return image

def load_and_preprocess_image(path, label):
    image = tf.io.read_file(path)
    return preprocess_image(image), label

In [ ]:
# Create TensorFlow datasets
train_dataset = tf.data.Dataset.from_tensor_slices((train_samples, train_labels))
train_dataset = train_dataset.map(load_and_preprocess_image)
train_dataset = train_dataset.shuffle(buffer_size=len(train_samples)).batch(32)

val_dataset = tf.data.Dataset.from_tensor_slices((val_samples, val_labels))
val_dataset = val_dataset.map(load_and_preprocess_image)
val_dataset = val_dataset.batch(32)

In [ ]:
# Define the EfficientNetB3 model architecture
base_model = EfficientNetB3(weights='imagenet', include_top=False)  # Update model architecture
x = base_model.output
x = GlobalAveragePooling2D()(x)
predictions = Dense(1, activation='sigmoid')(x)
model = Model(inputs=base_model.input, outputs=predictions)

In [ ]:
# Compile the model
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [ ]:
# Train the model
model.fit(train_dataset,
          epochs=5,
          validation_data=val_dataset)

Epoch 1/5
 95/313 [========>.....................] - ETA: 1:49 - loss: 0.2752 - accuracy: 0.8888

In [ ]:
# Save the model
model.save("efficientnet_b3_model.h5")

In [ ]:
# Evaluate the model
test_samples, test_labels = load_data(os.path.join(data_dir, "Test"), 2000)
test_dataset = tf.data.Dataset.from_tensor_slices((test_samples, test_labels))
test_dataset = test_dataset.map(load_and_preprocess_image)
test_dataset = test_dataset.batch(32)

loss, accuracy = model.evaluate(test_dataset)
print("Test Loss:", loss)
print("Test Accuracy:", accuracy)